In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Migration : AutoML Text Classification

<table align="left">
  <td>
<a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-text-classification-batch-prediction.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
<a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-text-classification-batch-prediction.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
      <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/migration/sdk-automl-text-classification-batch-prediction.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>   
</table>
<br/><br/><br/>

## Overview

This notebook demonstrates how to create an AutoML Text Classification Model, with a Vertex AI text dataset, and how to serve the model for batch prediction. It requires you provide a bucket where the dataset will be stored.

Note: you may incur charges for training, prediction, storage or usage of other GCP products in connection with testing this SDK.

Learn more about [Migrate to Vertex AI](https://cloud.google.com/vertex-ai/docs/start/migrating-to-vertex-ai) and [Classification for text data](https://cloud.google.com/vertex-ai/docs/training-overview#classification_for_text).

### Objective

The objective of this notebook is to build a AutoML Text Classification Model. The following steps have been followed:
This tutorial uses the following Google Cloud ML services :

* Vertex AI Dataset resource
* AutoML Training
* Vertex AI Model resource
* Vertex AI Batch Prediction

The steps performed include the following:

* Set your task name, and GCS prefix
* Copy AutoML text demo train data for creating managed dataset
* Create a dataset on Vertex AI.
* Configure a training job
* Launch a training job and create a model on Vertex AI
* Copy AutoML Text Demo Prediction Data for creating batch prediction job
* Perform batch prediction job on the model

### Dataset

The dataset used for this tutorial is the [Happy Moments dataset](https://www.kaggle.com/ritresearch/happydb) from [Kaggle Datasets](https://www.kaggle.com/ritresearch/happydb). The version of the dataset you will use in this tutorial is stored in a public Cloud Storage bucket.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the packages required for executing this notebook.

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 tensorflow 

### Colab only: Uncomment the following cell to restart the kernel

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.


In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [ ]:
import google.cloud.aiplatform as aip

## Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### Location of Cloud Storage training data.

Now set the variable `IMPORT_FILE` to the location of the CSV index file in Cloud Storage.

In [ ]:
IMPORT_FILE = "gs://cloud-ml-data/NL-classification/happiness.csv"

#### Quick peek at your data

This tutorial uses a version of the Happy Moments dataset that is stored in a public Cloud Storage bucket, using a CSV index file.

Start by doing a quick peek at the data. You count the number of examples by counting the number of rows in the CSV index file  (`wc -l`) and then peek at the first few rows.

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### Create the Dataset

### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

Next, create the `Dataset` resource using the `create` method for the `TextDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.
- `import_schema_uri`: The data labeling schema for the data items.

This operation may take several minutes.

In [ ]:
dataset = aip.TextDataset.create(
    display_name="Happy Moments" + "_" + UUID,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.text.single_label_classification,
)

print(dataset.resource_name)

*Example Output:*

    INFO:google.cloud.aiplatform.datasets.dataset:Creating TextDataset
    INFO:google.cloud.aiplatform.datasets.dataset:Create TextDataset backing LRO: projects/759209241365/locations/us-central1/datasets/3704325042721521664/operations/3193181053544038400
    INFO:google.cloud.aiplatform.datasets.dataset:TextDataset created. Resource name: projects/759209241365/locations/us-central1/datasets/3704325042721521664
    INFO:google.cloud.aiplatform.datasets.dataset:To use this TextDataset in another session:
    INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TextDataset('projects/759209241365/locations/us-central1/datasets/3704325042721521664')
    INFO:google.cloud.aiplatform.datasets.dataset:Importing TextDataset data: projects/759209241365/locations/us-central1/datasets/3704325042721521664
    INFO:google.cloud.aiplatform.datasets.dataset:Import TextDataset data backing LRO: projects/759209241365/locations/us-central1/datasets/3704325042721521664/operations/5152246891450204160
    INFO:google.cloud.aiplatform.datasets.dataset:TextDataset data imported. Resource name: projects/759209241365/locations/us-central1/datasets/3704325042721521664
    projects/759209241365/locations/us-central1/datasets/3704325042721521664

## Train a model

### [training.automl-api](https://cloud.google.com/vertex-ai/docs/training/automl-api)

### Create and run training pipeline

To train an AutoML model, you perform two steps: 1) create a training pipeline, and 2) run the pipeline.

#### Create training pipeline

An AutoML training pipeline is created with the `AutoMLTextTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `prediction_type`: The type task to train the model for.
  - `classification`: A text classification model.
  - `sentiment`: A text sentiment analysis model.
  - `extraction`: A text entity extraction model.
- `multi_label`: If a classification task, whether single (False) or multi-labeled (True).
- `sentiment_max`: If a sentiment analysis task, the maximum sentiment value.

The instantiated object is the DAG (directed acyclic graph) for the training pipeline.

In [ ]:
dag = aip.AutoMLTextTrainingJob(
    display_name="happydb_" + UUID,
    prediction_type="classification",
    multi_label=False,
)

print(dag)

*Example output:*

    <google.cloud.aiplatform.training_jobs.AutoMLTextTrainingJob object at 0x7fc3b6c90f10>

#### Run the training pipeline

Next, you run the DAG to start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `validation_fraction_split`: The percentage of the dataset to use for validation.

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline will take up a few hours.

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="happydb_" + UUID,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

*Example output:*

    INFO:google.cloud.aiplatform.training_jobs:View Training:
    https://console.cloud.google.com/ai/platform/locations/us-central1/training/8859754745456230400?project=759209241365
    INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/8859754745456230400 current state:
    PipelineState.PIPELINE_STATE_RUNNING
    INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/8859754745456230400 current state:
    PipelineState.PIPELINE_STATE_RUNNING
    INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/8859754745456230400 current state:
    PipelineState.PIPELINE_STATE_RUNNING
    INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/8859754745456230400 current state:
    PipelineState.PIPELINE_STATE_RUNNING
    INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/8859754745456230400 current state:
    PipelineState.PIPELINE_STATE_RUNNING
    ...
    INFO:google.cloud.aiplatform.training_jobs:AutoMLTextTrainingJob run completed. Resource name: projects/759209241365/locations/us-central1/trainingPipelines/8859754745456230400
    INFO:google.cloud.aiplatform.training_jobs:Model available at projects/759209241365/locations/us-central1/models/6389525951797002240

## Review model evaluation scores
After your model has finished training, you can review the evaluation scores for it.

First, you need to get a reference to the new model. As with datasets, you can either use the reference to the model variable you created when you deployed the model or you can list all of the models in your project.

### [projects.locations.models.evaluations.list](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=happydb_" + UUID)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

*Example output:*

    name: "projects/759209241365/locations/us-central1/models/623915674158235648/evaluations/4280507618583117824"
    metrics_schema_uri: "gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml"
    metrics {
      struct_value {
        fields {
          key: "auPrc"
          value {
            number_value: 0.9891107
          }
        }
        fields {
          key: "confidenceMetrics"
          value {
            list_value {
              values {
                struct_value {
                  fields {
                    key: "precision"
                    value {
                      number_value: 0.2
                    }
                  }
                  fields {
                    key: "recall"
                    value {
                      number_value: 1.0
                    }
                  }
                }
              }

## Make batch predictions

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

### Get test item(s)

Now do a batch prediction to your Vertex model. You will use arbitrary examples out of the dataset as a test items. Don't be concerned that the examples were likely used in training the model -- you just want to demonstrate how to make a prediction.

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2
if len(test_items[0]) == 3:
    _, test_item_1, test_label_1 = str(test_items[0]).split(",")
    _, test_item_2, test_label_2 = str(test_items[1]).split(",")
else:
    test_item_1, test_label_1 = str(test_items[0]).split(",")
    test_item_2, test_label_2 = str(test_items[1]).split(",")

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### Make the batch input file

Now make a batch input file, which you will store in your local Cloud Storage bucket. The batch input file can only be in JSONL format. For JSONL file, you make one dictionary entry per line for each data item (instance). The dictionary contains the key/value pairs:

- `content`: The Cloud Storage path to the file with the text item.
- `mime_type`: The content type. In our example, it is a `text` file.

For example:

                        {'content': '[your-bucket]/file1.txt', 'mime_type': 'text'}

In [ ]:
import json
import os

import tensorflow as tf

gcs_test_item_1 = BUCKET_URI + "/test1.txt"
with tf.io.gfile.GFile(gcs_test_item_1, "w") as f:
    f.write(test_item_1 + "\n")
gcs_test_item_2 = BUCKET_URI + "/test2.txt"
with tf.io.gfile.GFile(gcs_test_item_2, "w") as f:
    f.write(test_item_2 + "\n")

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": gcs_test_item_1, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")
    data = {"content": gcs_test_item_2, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### Make the batch prediction request

Now that your Model resource is trained, you can make a batch prediction by invoking the batch_predict() method, with the following parameters:

- `job_display_name`: The human readable name for the batch prediction job.
- `gcs_source`: A list of one or more batch request input files.
- `gcs_destination_prefix`: The Cloud Storage location for storing the batch prediction resuls.
- `sync`: If set to True, the call will block while waiting for the asynchronous batch job to complete.

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="happydb_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

*Example output:*

    INFO:google.cloud.aiplatform.jobs:Creating BatchPredictionJob
    <google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7f806a6112d0> is waiting for upstream dependencies to complete.
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296
    INFO:google.cloud.aiplatform.jobs:To use this BatchPredictionJob in another session:
    INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296')
    INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5110965452507447296?project=759209241365
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296 current state:
    JobState.JOB_STATE_RUNNING

### Wait for completion of batch prediction job

Next, wait for the batch job to complete. Alternatively, one can set the parameter `sync` to `True` in the `batch_predict()` method to block until the batch prediction job is completed.

In [ ]:
batch_predict_job.wait()

*Example Output:*

    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328
    INFO:google.cloud.aiplatform.jobs:To use this BatchPredictionJob in another session:
    INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328')
    INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/181835033978339328?project=759209241365
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_SUCCEEDED
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob run completed. Resource name: projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328

### Get the predictions

Next, get the results from the completed batch prediction job.

The results are written to the Cloud Storage output bucket you specified in the batch prediction request. You call the method iter_outputs() to get a list of each Cloud Storage file generated with the results. Each file contains one or more prediction requests in a JSON format:

- `content`: The prediction request.
- `prediction`: The prediction response.
 - `ids`: The internal assigned unique identifiers for each prediction request.
 - `displayNames`: The class names for each class label.
 - `confidences`: The predicted confidence, between 0 and 1, per class label.

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

*Example Output:*

    {'instance': {'content': 'gs://andy-1234-221921aip-20210803210202/test2.txt', 'mimeType': 'text/plain'}, 'prediction': {'ids': ['681905016918769664', '3564208778435887104', '8175894796863275008', '5538192790107717632', '5870051787649581056', '3232349780894023680', '926506771680329728'], 'displayNames': ['affection', 'achievement', 'bonding', 'enjoy_the_moment', 'nature', 'leisure', 'exercise'], 'confidences': [0.9977309, 0.0017838771, 0.0002530971, 0.00014939539, 4.747714e-05, 2.6297073e-05, 8.965492e-06]}}

## Make online predictions

### [predictions.deploy-model-api](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

## Deploy the model

Next, deploy your model for online prediction. To deploy the model, you invoke the `deploy` method.

In [ ]:
endpoint = model.deploy()

*Example output:*

    INFO:google.cloud.aiplatform.models:Creating Endpoint
    INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/4087251132693348352
    INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
    INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/759209241365/locations/us-central1/endpoints/4867177336350441472')
    INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/1691336130932244480
    INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/759209241365/locations/us-central1/endpoints/4867177336350441472

## Online prediction of automl
### [predictions.online-prediction-automl](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)

### Get test item

You will use an arbitrary example out of the dataset as a test item. Don't be concerned that the example was likely used in training the model -- you just want to demonstrate how to make a prediction.

In [ ]:
test_item = ! gsutil cat $IMPORT_FILE | head -n1
if len(test_item[0]) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

### Make the prediction

Now that your `Model` resource is deployed to an `Endpoint` resource, you can do online predictions by sending prediction requests to the `Endpoint` resource.

#### Request

The format of each instance is:

     { 'content': text_string }

Since the predict() method can take multiple items (instances), send your single test item as a list of one test item.

#### Response

The response from the predict() call is a Python dictionary with the following entries:

- `ids`: The internal assigned unique identifiers for each prediction request.
- `displayNames`: The class names for each class label.
- `confidences`: The predicted confidence, between 0 and 1, per class label.
- `deployed_model_id`: The Vertex AI identifier for the deployed `Model` resource which did the predictions.

In [ ]:
instances_list = [{"content": test_item}]

prediction = endpoint.predict(instances_list)
print(prediction)

*Example output:*

    Prediction(predictions=[{'confidences': [0.9807877540588379, 0.0029202369041740894, 0.001903864904306829, 0.013396155089139938, 0.0002868965675588697, 0.00017845185357145965, 0.0005265594809316099], 'displayNames': ['affection', 'achievement', 'enjoy_the_moment', 'bonding', 'leisure', 'nature', 'exercise'], 'ids': ['1022137895017775104', '6210284665748586496', '3327980904231469056', '8516127674962280448', '7939666922658856960', '3904441656534892544', '5633823913445163008']}], deployed_model_id='8719822099612434432', explanations=None)
affection

## Undeploy the model

When you are done doing predictions, you undeploy the model from the `Endpoint` resouce. This deprovisions all compute resources and ends billing for the deployed model.

In [ ]:
endpoint.undeploy_all()

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the endpoint using the Vertex endpoint object
endpoint.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

# Delete the bucket
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI